<!--<badge>--><a href="https://colab.research.google.com/github/gradsflow/gradsflow/blob/main/examples/nbs/05-model_fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

In [ ]:
import torchvision
from timm import create_model
from torch.utils.data import DataLoader
from torchvision import transforms as T

from gradsflow import AutoDataset, Model
from gradsflow.callbacks import (
    CSVLogger,
    EmissionTrackerCallback,
    ModelCheckpoint,
)
from gradsflow.data.common import random_split_dataset

In [ ]:
# Replace dataloaders with your custom dataset and you are all set to train your model
image_size = (64, 64)
batch_size = 4

to_rgb = lambda x: x.convert("RGB")

augs = T.Compose([to_rgb, T.AutoAugment(), T.Resize(image_size), T.ToTensor()])
data = torchvision.datasets.Caltech101("~/", download=True, transform=augs)
train_data, val_data = random_split_dataset(data, 0.99)
train_dl = DataLoader(train_data, batch_size=batch_size)
val_dl = DataLoader(val_data, batch_size=batch_size)
num_classes = len(data.categories)

Files already downloaded and verified


In [ ]:
cbs = [
    CSVLogger(
        verbose=True,
    ),
    ModelCheckpoint(),
    EmissionTrackerCallback(),
    # CometCallback(offline=True),
]

CODECARBON : No CPU tracking mode found. Falling back on CPU constant mode.
/Users/aniket/miniconda3/envs/am/lib/python3.9/site-packages/apscheduler/util.py:95: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  if obj.zone == 'local':
/Users/aniket/miniconda3/envs/am/lib/python3.9/site-packages/apscheduler/triggers/interval.py:66: PytzUsageWarning: The normalize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return self.timezone.normalize(next_fire_time)


In [ ]:
autodataset = AutoDataset(train_dl, val_dl, num_classes=num_classes)
cnn = create_model("resnet18", pretrained=False, num_classes=num_classes)

model = Model(cnn)

model.compile("crossentropyloss", "adam", metrics=["accuracy"])
model.fit(autodataset, max_epochs=10, steps_per_epoch=10, callbacks=cbs)

Output()